<a href="https://colab.research.google.com/github/skssushil/Web-mining/blob/master/1_4Represent_collection_of_documents_in_VSM_using_tf_and_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import nltk
import string
import numpy as np
import pandas as pd
import re
import math
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import io
with open('CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")

In [3]:
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

In [4]:
dataset = []
for i in doc_set.values():
  dataset.append(i)

In [5]:
N=len(dataset)
print(N)

1460


In [6]:
import io
with open('CISI.QRY') as g:
  qlines = ""
  for m in g.readlines():
    qlines += "\n" + m.strip() if m.startswith(".") else " " + m.strip()
  qlines = qlines.lstrip("\n").split("\n")

In [7]:
qry_set = {}
qry_id = ""
qry_text=""
for m in qlines:
  if m.startswith(".I"):
    qry_id = m.split(" ")[1].strip()
  elif m.startswith(".W"):
    qry_set[qry_id] = m.strip()[3:]
    qry_id = ""
squery=qry_set["1"]

In [8]:
def convert_lower_case(data):
  return np.char.lower(data)

In [9]:
def remove_stop_words(data):
  stop_words = stopwords.words('english')
  words = word_tokenize(str(data))
  new_text = ""
  for w in words:
    if w not in stop_words and len(w) > 1:
      new_text = new_text + " " + w
  return new_text

In [10]:
def remove_punctuation(data):
  symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
  for i in range(len(symbols)):
    data = np.char.replace(data, symbols[i], ' ')
    data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
  return data

In [11]:
def remove_apostrophe(data):
  return np.char.replace(data, "'", "")

In [12]:
def stemming(data):
  stemmer= PorterStemmer()  
  tokens = word_tokenize(str(data))
  new_text = ""
  for w in tokens:
    new_text = new_text + " " + stemmer.stem(w)
  return new_text

In [13]:
import inflect 
p = inflect.engine()   
# convert number into words 
def convert_number(text): 
  temp_str = text.split() 
  new_string = []  
  for word in temp_str:
    if word.isdigit(): 
      temp = p.number_to_words(word) 
      new_string.append(temp) 
    else:
      new_string.append(word) 
      temp_str = ' '.join(new_string) 
  return temp_str

In [14]:
def preprocess(data):
  data = convert_number(data)
  data = convert_lower_case(data)
  data = remove_punctuation(data) #remove comma seperately
  data = remove_apostrophe(data)
  data = remove_stop_words(data)
  data = stemming(data)
  data = remove_punctuation(data)
  data = stemming(data) #needed again as we need to stem the words
  data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
  data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
  data = convert_number(data)
  return data

In [15]:
processed_text = []
for i in dataset[:N]:
  processed_text.append(word_tokenize(str(preprocess(i))))

In [16]:
DF = {}
for i in range(N):
  tokens = processed_text[i]
  for w in tokens:
    try:
      DF[w].add(i)
    except:
      DF[w] = {i}
for i in DF:
  DF[i] = len(DF[i])

In [17]:
total_vocab_size = len(DF)
print(total_vocab_size)

6830


In [18]:
total_vocab = [x for x in DF]

In [19]:
print(total_vocab[:20])

['eighteen', 'edit', 'dewey', 'decim', 'classif', 'comaromi', 'present', 'studi', 'histori', 'first', 'ddc', 'publish', 'one', 'thousand', ',', 'eight', 'hundred', 'and', 'seventy-six', 'eighteenth']


In [20]:
def doc_freq(word):
  c = 0
  try:
    c = DF[word]
  except:
    pass
  return c

In [21]:
doc = 0
tf_idf = {}
for i in range(N):    
    tokens = processed_text[i]    
    counter = Counter(tokens)
    words_count = len(tokens)    
    for token in np.unique(tokens):        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))        
        tf_idf[doc, token] = tf*idf
    doc += 1

In [22]:
doc = 0
tf_idf = {}
for i in range(N):    
    tokens = processed_text[i]    
    counter = Counter(tokens)
    words_count = len(tokens)    
    for token in np.unique(tokens):        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))        
        tf_idf[doc, token] = tf*idf
    doc += 1

In [29]:
tf_idf

{(0, ','): 0.06234496611467591,
 (0, 'abroad'): 0.08722407845274038,
 (0, 'and'): 0.060539017720959994,
 (0, 'appear'): 0.044777274493589134,
 (0, 'attempt'): 0.03889832547300351,
 (0, 'biographi'): 0.09366003254969538,
 (0, 'briefli'): 0.05923060873430612,
 (0, 'classif'): 0.08328372436948038,
 (0, 'comaromi'): 0.10466236874905958,
 (0, 'continu'): 0.048456665192911745,
 (0, 'countri'): 0.05455125095302604,
 (0, 'ddc'): 0.17444815690548077,
 (0, 'decim'): 0.1413861291331582,
 (0, 'describ'): 0.026683719769122254,
 (0, 'detail'): 0.04225243807089569,
 (0, 'dewey'): 0.22132471819692573,
 (0, 'edit'): 0.22096596330034154,
 (0, 'eight'): 0.04371868124757847,
 (0, 'eighteen'): 0.07165536015096695,
 (0, 'eighteenth'): 0.10466236874905958,
 (0, 'first'): 0.06718706084801739,
 (0, 'full'): 0.056336615769322716,
 (0, 'futur'): 0.04321473365528354,
 (0, 'growth'): 0.048688392167850675,
 (0, 'healthi'): 0.10466236874905958,
 (0, 'histori'): 0.10528839676820881,
 (0, 'howev'): 0.04076789746754928

In [30]:
len(tf_idf)

81189

In [31]:
def matching_score(k, query):
  preprocessed_query = preprocess(query)
  tokens = word_tokenize(str(preprocessed_query))
  print("\nQuery:", query)  
  query_weights = {}
  for key in tf_idf:
    if key[1] in tokens:
      try:
        query_weights[key[0]] += tf_idf[key]
      except:
        query_weights[key[0]] = tf_idf[key]    
  query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
  print("")  
  print("Top Matching Documents")  
  l = []
  for i in query_weights[:10]:
    l.append(i[0])    
  print(l)
  for i in range(k):
    x=l[i]
    print("    ", l[i]," ", dataset[x][:100])

In [32]:
matching_score(10, squery)


Query: What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?

Top Matching Documents
[1293, 448, 446, 314, 1280, 443, 428, 564, 721, 1162]
     1293   Automatic Text Analysis Salton, G. In this article the principal experiments in automatic text analy
     448   Situational Relevance Wilson, Patrick The concept of situational relevance is introduced, based on W
     446   A Note on the Concept of "Relevance" Foskett, D.J. Two recent articles in this journal (Konigova [1]
     314   Automatic Abstracting and Indexing - Survey and Recommendations Edmundson, H.P. Wyllys, R.E. In prep
     1280   Relative Effectiveness of Document Titles and Abstracts for Determining Relevance of Documents Resni
     443   On Relevance as a Measure Coffman, William Relevance is defined as a measure of information conveyed
     

In [33]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [34]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [35]:
def gen_vector(tokens):
    Q = np.zeros((len(total_vocab)))
    counter = Counter(tokens)
    words_count = len(tokens)
    query_weights = {}
    for token in np.unique(tokens):   
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))
        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [36]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    print("\nQuery:", query)
    print("")
    #print(tokens)
    d_cosines = []
    query_vector = gen_vector(tokens)
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
    out = np.array(d_cosines).argsort()[-k:][::-1]
    print("")  
    print(out)
    print("Cosine similarity"," id ", " Document")
    for i in range(k):
      x=out[i]
      print(round(d_cosines[x],10),"    ", out[i]," ", dataset[x][:100])


In [37]:
cosine_similarity(10,squery)

Cosine Similarity

Query: What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?


[ 721  428  588  602 1280  812 1298  710  835   37]
Cosine similarity  id   Document
0.3699350395      721   Information Transfer Limitations of Titles of Chemical Documents Bottle, R.R. Seeley, C.R. Some meth
0.3395118108      428   The Information Content of Titles in Engineering Literature Bottle, Robert T. Since many alerting an
0.2880003582      588   Are Titles of Chemical Papers Becoming More Informative? Tocatlian, Jacques J. The efficiency of key
0.2641313807      602   The Efficiency of MEDLARS Titles for Retrieval Miller, William L. Previous research has indicated th
0.2525271318      1280   Relative Effectiveness of Document Titles and Abstracts for Determining Relevance of Documents Resni
0.2126737211      812   Co